In [25]:
import pandas as pd
import numpy as np
import os
import warnings
from scipy.stats import nbinom, gamma, fisher_exact
from scipy.special import digamma
from scipy.optimize import minimize


In [26]:
def download_faer_files(root_dir):
    # Define root directory where FAERS datasets are located
    quarters = ['Q1', 'Q2', 'Q3', 'Q4']

    # Adjust range from 2013 to 2025
    all_quarters = [f"{str(y)[2:]}{q}" for y in range(2017, 2025) for q in quarters]

    # Limit to only those quarters that exist (sanity check)
    existing_quarters = [q for q in all_quarters if os.path.exists(os.path.join(root_dir, f"DEMO{q}.txt"))]

    # Store DataFrames in a dict
    data = {}

    for q in existing_quarters:
        try:
            data[q] = {
                'demo': pd.read_csv(os.path.join(root_dir, f"DEMO{q}.txt"), delimiter='$', encoding='ISO-8859-1'),
                'drug': pd.read_csv(os.path.join(root_dir, f"DRUG{q}.txt"), delimiter='$', encoding='ISO-8859-1'),
                'reac': pd.read_csv(os.path.join(root_dir, f"REAC{q}.txt"), delimiter='$', encoding='ISO-8859-1'),
                'outc': pd.read_csv(os.path.join(root_dir, f"OUTC{q}.txt"), delimiter='$', encoding='ISO-8859-1'),
                'indi': pd.read_csv(os.path.join(root_dir, f"INDI{q}.txt"), delimiter='$', encoding='ISO-8859-1'),
                'rpsr': pd.read_csv(os.path.join(root_dir, f"RPSR{q}.txt"), delimiter='$', encoding='ISO-8859-1'),
                'ther': pd.read_csv(os.path.join(root_dir, f"THER{q}.txt"), delimiter='$', encoding='ISO-8859-1'),
            }
            print(f"Loaded {q} successfully.")
        except Exception as e:
            print(f"Error loading data for {q}: {e}")

    return data



In [27]:
def generate_periods(start_year, start_quarter, end_year, end_quarter):
    periods = []
    quarters = ['Q1', 'Q2', 'Q3', 'Q4']

    start_year = int(start_year)
    end_year = int(end_year)

    # Loop through the years and quarters to generate all periods in the range
    for year in range(start_year, end_year + 1):
        start_qtr = start_quarter if year == start_year else 'Q1'
        end_qtr = end_quarter if year == end_year else 'Q4'

        for qtr in quarters[quarters.index(start_qtr):quarters.index(end_qtr) + 1]:
            periods.append(f"{str(year)[-2:]}{qtr}")
    return periods


In [28]:
def create_dataframes(start_year, start_quarter, end_year, end_quarter):
    # Generate periods based on user input
    periods = generate_periods(start_year, start_quarter, end_year, end_quarter)
    print(periods)

    # List of table types to be processed 
    table_types = ['demo', 'drug', 'reac', 'outc', 'indi', 'rpsr', 'ther']

    # Initialize a dictionary to store lists of DataFrames for each table
    data_dict = {table: [] for table in table_types}

    # Loop through each period and collect data for each table type
    for period in periods:
        if period in data:  # Check if data for the period exists
            for table in table_types:
                if table in data[period]:
                    data_dict[table].append(data[period][table])
        else:
            print(f"Warning: No data available for {period}")

    # Concatenate data for each table type into a single DataFrame
    merged_data = {
        table: pd.concat(data_dict[table], ignore_index=True) if data_dict[table] else pd.DataFrame()
        for table in table_types
    }

    demo = merged_data['demo']
    drug = merged_data['drug']
    reac = merged_data['reac']
    outc = merged_data['outc']
    indi = merged_data['indi']
    rpsr = merged_data['rpsr']
    ther = merged_data['ther']

    return demo, drug, reac, outc, indi, rpsr, ther


In [29]:
def preprocess_drug_df(drug):
    drug = drug[['primaryid', 'caseid', 'role_cod', 'drugname', 'prod_ai']]
    drug = drug[drug['role_cod'] == 'PS']

    drug = drug[pd.notnull(drug['drugname'])]  # Drops Nulls
    drug['drugname'] = drug['drugname'].str.strip().str.lower()  # Stips whitespace, Transforms to lowercase
    drug = drug[~drug['drugname'].isin(['unknown'])]  # Drops unknowns
    drug['drugname'] = drug['drugname'].str.replace('\\', '/')  # Standardizes slashes to '/'
    drug['drugname'] = drug['drugname'].map(
        lambda x: x[:-1] if str(x).endswith(".") else x)  # Removes periods at the end of drug names

    return drug


In [30]:
def preprocess_reac_df(reac):
    reac = reac[pd.notnull(reac['pt'])] # Drops Nulls

    reac['pt'] = reac['pt'].str.strip().str.lower()  # Transforms to lowercase
    reac = reac[~reac['pt'].isin(['unknown'])]  # Drops unknowns
    reac['pt'] = reac['pt'].map(
        lambda x: x[:-1] if str(x).endswith(".") else x)  # Removes periods at the end of drug names

    return reac


In [31]:
def preprocess_demo_df(demo):
    demo = demo[['primaryid', 'caseid', 'fda_dt', 'caseversion', 'age_cod', 'age', 'sex', 'wt']]
    demo = demo.sort_values(by=['caseid', 'fda_dt', 'primaryid'], ascending=[True, False, False])
    demo = demo.drop_duplicates(subset=['caseid'], keep='first')

    """
    demo = demo[pd.notnull(demo['age']) 
    demo = demo[demo.age_cod != 'dec'].reset_index(drop=True)
    demo['age'] = demo['age'].apply(pd.to_numeric, errors='coerce')
    demo['age'] = np.where(demo['age_cod'] == 'MON', demo['age'] * 1 / 12, demo['age'])  # mounth
    demo['age'] = np.where(demo['age_cod'] == 'WK', demo['age'] * 1 / 52, demo['age'])  # week
    demo['age'] = np.where(demo['age_cod'] == 'DY', demo['age'] * 1 / 365, demo['age'])  # day
    demo['age'] = np.where(demo['age_cod'] == 'HR', demo['age'] * 1 / 8760, demo['age'])  # hour
    demo = demo.drop(['age_cod'], axis=1)
    """ 

    return demo


In [32]:
def phv_ebgm_qn(theta_hat, N, E):
    r1, b1, r2, b2, p = theta_hat
    prob_f1 = b1 / (b1 + E)
    prob_f2 = b2 / (b2 + E)
    f1_nb = nbinom.pmf(N, n=r1, p=prob_f1)
    f2_nb = nbinom.pmf(N, n=r2, p=prob_f2)
    num = p * f1_nb
    den = num + (1 - p) * f2_nb
    return num / den

def phv_ebgm_score(theta_hat, N, E, qn):
    r1, b1, r2, b2, _ = theta_hat
    e1 = digamma(r1 + N) - np.log(b1 + E)
    e2 = digamma(r2 + N) - np.log(b2 + E)
    exp_log = qn * e1 + (1 - qn) * e2
    eb_log2 = exp_log / np.log(2)
    return 2 ** eb_log2

def phv_ebgm_quant_bisect(cut_point, theta_hat, N, E, qn,
                          digits=2, limits=(-1e5, 1e5), max_iter=2000):
    r1, b1, r2, b2, _ = theta_hat
    lower, upper = limits
    tol = 0.5 * 10 ** (-digits)

    def post_cdf_minus_cp(x):
        c1 = gamma.cdf(x, a=r1 + N, scale=1/(b1 + E))
        c2 = gamma.cdf(x, a=r2 + N, scale=1/(b2 + E))
        return qn * c1 + (1 - qn) * c2 - cut_point

    N = np.asarray(N)
    E = np.asarray(E)
    qn = np.asarray(qn)

    guess = np.ones_like(N, dtype=float)
    err0 = post_cdf_minus_cp(guess)
    is_pos = err0 > 0
    left = np.where(is_pos, lower, guess)
    right = np.where(is_pos, guess, upper)

    for _ in range(max_iter):
        mid = (left + right) / 2
        err_mid = post_cdf_minus_cp(mid)
        if np.max((right - left) / 2) < tol:
            q = np.round(mid, digits)
            if np.any(q == upper):
                raise ValueError("increase maximum for 'limits'")
            return q
        err_left = post_cdf_minus_cp(left)
        same = np.sign(err_left) == np.sign(err_mid)
        left = np.where(same, mid, left)
        right = np.where(same, right, mid)

    raise RuntimeError("failed to converge -- try adjusting 'limits' or 'max_iter'")

def dbinbinom(x, size1, prob1, size2, prob2, w):
    return w * nbinom.pmf(x, n=size1, p=prob1) + (1 - w) * nbinom.pmf(x, n=size2, p=prob2)

def phvid_objective(theta, N, E):
    r1, b1, r2, b2, w = theta
    prob1 = b1 / (b1 + E)
    prob2 = b2 / (b2 + E)
    pmf = dbinbinom(N, r1, prob1, r2, prob2, w)
    return np.sum(-np.log(pmf + 1e-16))  # add tiny epsilon to avoid log(0)

def phv_ebgm(a, b, c, d, alpha=0.05, theta_init=None, squashing=True):
    """
    a, b, c, d: counts (scalars or array‐like)
    alpha: two‐sided significance level
    theta_init: array‐like [r1, b1, r2, b2, p] or pandas.DataFrame of guesses
    squashing: (not implemented – placeholder)
    """
    # turn inputs into arrays
    a, b, c, d = np.broadcast_arrays(a, b, c, d)
    if np.any(a < 0) or np.any(b < 0) or np.any(c < 0) or np.any(d < 0):
        raise ValueError("a, b, c, d must be non‐negative")
    # total counts
    tot = a + b + c + d
    N = a.astype(float)
    E = (a + b) / tot * (a + c)

    # squashing placeholder
    if squashing and np.any(a == 0):
        warnings.warn("squashing=True but no squash implemented; continuing with raw counts")

    # prepare initial guess
    if theta_init is None:
        x0 = np.array([0.2, 0.1, 2.0, 4.0, 1/3])
    else:
        if isinstance(theta_init, pd.DataFrame):
            x0 = theta_init.iloc[0].values
        else:
            x0 = np.asarray(theta_init, dtype=float)
        if x0.shape[0] != 5:
            raise ValueError("theta_init must have length 5")

    # bounds: r1,b1,r2,b2 > 0; p in (0,1)
    bounds = [(1e-6, None)]*4 + [(1e-6, 1-1e-6)]
    res = minimize(phvid_objective, x0, args=(N, E),
                   bounds=bounds, method='L-BFGS-B')
    if not res.success:
        raise RuntimeError("hyperparameter estimation failed: " + res.message)
    theta_hat = res.x

    # compute posterior weights, EBGM and CIs
    qn = phv_ebgm_qn(theta_hat, N, E)
    ebgm = phv_ebgm_score(theta_hat, N, E, qn)
    half = alpha / 2
    ci_low  = phv_ebgm_quant_bisect(half,      theta_hat, N, E, qn)
    ci_high = phv_ebgm_quant_bisect(1 - half,  theta_hat, N, E, qn)

    return pd.DataFrame({
        'ebgm':   ebgm,
        'ci_low': ci_low,
        'ci_high':ci_high
    })

In [50]:
def compute_or_and_ci(a, b, c, d):
    """
    Computes Odds Ratio (OR) and Confidence Interval (CI)
    """
    # Odds Ratio
    or_val = (a * d) / (b * c)

    # Log(OR) and standard error
    log_or = np.log(or_val)
    se = np.sqrt(1 / a + 1 / b + 1 / c + 1 / d)

    # Confidence Interval
    ci_low = np.exp(log_or - 1.96 * se)
    ci_high = np.exp(log_or + 1.96 * se)

    return or_val, ci_low, ci_high


def compute_prr_and_ci(a, b, c, d):
    """ 
    Computes Proportional Reporting Ratio (PRR), Standard Error (SE), and Confidence Interval (CI)
    """ 
    # Proportional Reporting Ratio (PRR)
    prr = (a / (a + b)) / (c / (c + d))

    # Standard Error (SE)
    se = np.sqrt(1 / a + 1 / c - 1 / (a + b) - 1 / (c + d))

    # Confidence Interval for PRR
    ln_prr = np.log(prr)
    ci_low = np.exp(ln_prr - 1.96 * se)
    ci_high = np.exp(ln_prr + 1.96 * se)

    return prr, se, ci_low, ci_high

def compute_bayesian_ic(a, b, c, d,
                        gamma_11=0.5, alpha1=0.5, beta1=0.5, 
                        alpha=2.0, beta=2.0):
    """
    Compute Bayesian Information Component (IC), E(IC), V(IC), and IC-2SD
    based on counts a, b, c, d and prior hyperparameters.
    
    Parameters:
    - a, b, c, d: cell counts (can be arrays)
    - gamma_11, alpha1, beta1, alpha, beta: prior pseudo-counts (defaults are Jeffrey's prior)
    
    Returns:
    - DataFrame with E(IC), Var(IC), IC_2SD (lower bound)
    """
    # Total N
    N = a + b + c + d
    
    # Compute gamma (normalization term)
    gamma = gamma_11 * ((N + alpha) * (N + beta)) / ((a + b + alpha1) * (a + c + beta1))

    # Expected value of IC (E(IC))
    E_ic = np.log2(
        (a + gamma_11) * (N + alpha) * (N + beta) /
        ((N + gamma) * (a + b + alpha1) * (a + c + beta1))
    )

    # Compute variance of IC (V(IC))
    ln2_sq = np.log(2) ** 2
    term1 = ((N - a + gamma - gamma_11) / ((a + gamma_11) * (1 + N + gamma)))
    term2 = ((N - (a + b) + alpha - alpha1) / ((a + b + alpha1) * (1 + N + alpha)))
    term3 = ((N - (a + c) + beta - beta1) / ((a + c + beta1) * (1 + N + beta)))
    V_ic = (1 / ln2_sq) * (term1 + term2 + term3)

    # Lower bound: IC - 2 * SD
    IC_2SD = E_ic - 2 * np.sqrt(V_ic)

    return pd.DataFrame({
        "E_IC": E_ic,
        "Var_IC": V_ic,
        "IC_2SD": IC_2SD
    })

In [52]:
# Function to add statistics (OR, PRR, EGBM) to the DataFrame
def add_stats(df):
    a = df['Count_query_drug'] 
    b = df['No_AE_query_drug'] 
    c = df['Count_non_query_drug'] 
    d = df['No_AE_non_query_drug'] 

    # Compute Odds Ratio (OR) and Confidence Interval (CI) 
    or_vals, ci_low_or, ci_high_or = compute_or_and_ci(a, b, c, d)

    # Compute Proportional Reporting Ratio (PRR), SE, and Confidence Interval (CI) 
    prr_vals, se_prr, ci_low_prr, ci_high_prr = compute_prr_and_ci(a, b, c, d)

    # Compute p-value from fischer's exact test for each row (used due to small counts) 
    p_values = df.apply(
        lambda row: fisher_exact([[row['Count_query_drug'], row['No_AE_query_drug']],
                              [row['Count_non_query_drug'], row['No_AE_non_query_drug']]])[1], axis=1
        )

    # Compute ebgm statistic 
    ebgm_result = phv_ebgm(a, b, c, d, alpha=0.5)

    # Compute ic statistic 
    ic_result = compute_bayesian_ic(a, b, c, d, gamma_11=0.5, alpha1=0.5, beta1=0.5, alpha=2.0, beta=2.0)

    
    # Add new columns for OR, PRR, and p-value statistics
    df['odds_ratio'] = or_vals
    df['ci_lower_or'] = ci_low_or
    df['ci_upper_or'] = ci_high_or
    df['prr'] = prr_vals
    df['se_prr'] = se_prr
    df['ci_lower_prr'] = ci_low_prr
    df['ci_upper_prr'] = ci_high_prr
    df['p_value'] = p_values
    
    df = df.merge(ebgm_result, left_index=True, right_index=True)
    df = df.merge(ic_result, left_index=True, right_index=True)

    print(df)
    return df



In [35]:
root_dir = '/Users/jodie/Documents/BMI 212/faers-cohort-generation/FAERS-data-toolkit-master/FAERSdata'
data = download_faer_files(root_dir)


/var/folders/1z/lz8g1zg14zv6dkhlyvsk7zvc0000gn/T/ipykernel_10022/1006456903.py:17: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  'demo': pd.read_csv(os.path.join(root_dir, f"DEMO{q}.txt"), delimiter='$', encoding='ISO-8859-1'),
/var/folders/1z/lz8g1zg14zv6dkhlyvsk7zvc0000gn/T/ipykernel_10022/1006456903.py:18: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  'drug': pd.read_csv(os.path.join(root_dir, f"DRUG{q}.txt"), delimiter='$', encoding='ISO-8859-1'),
/var/folders/1z/lz8g1zg14zv6dkhlyvsk7zvc0000gn/T/ipykernel_10022/1006456903.py:23: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  'ther': pd.read_csv(os.path.join(root_dir, f"THER{q}.txt"), delimiter='$', encoding='ISO-8859-1'),


Loaded 17Q1 successfully.


/var/folders/1z/lz8g1zg14zv6dkhlyvsk7zvc0000gn/T/ipykernel_10022/1006456903.py:17: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  'demo': pd.read_csv(os.path.join(root_dir, f"DEMO{q}.txt"), delimiter='$', encoding='ISO-8859-1'),
/var/folders/1z/lz8g1zg14zv6dkhlyvsk7zvc0000gn/T/ipykernel_10022/1006456903.py:18: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  'drug': pd.read_csv(os.path.join(root_dir, f"DRUG{q}.txt"), delimiter='$', encoding='ISO-8859-1'),
/var/folders/1z/lz8g1zg14zv6dkhlyvsk7zvc0000gn/T/ipykernel_10022/1006456903.py:23: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  'ther': pd.read_csv(os.path.join(root_dir, f"THER{q}.txt"), delimiter='$', encoding='ISO-8859-1'),


Loaded 17Q2 successfully.


/var/folders/1z/lz8g1zg14zv6dkhlyvsk7zvc0000gn/T/ipykernel_10022/1006456903.py:18: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  'drug': pd.read_csv(os.path.join(root_dir, f"DRUG{q}.txt"), delimiter='$', encoding='ISO-8859-1'),


...

Loaded 24Q2 successfully.


/var/folders/1z/lz8g1zg14zv6dkhlyvsk7zvc0000gn/T/ipykernel_10022/1006456903.py:17: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  'demo': pd.read_csv(os.path.join(root_dir, f"DEMO{q}.txt"), delimiter='$', encoding='ISO-8859-1'),


Loaded 24Q3 successfully.


/var/folders/1z/lz8g1zg14zv6dkhlyvsk7zvc0000gn/T/ipykernel_10022/1006456903.py:17: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  'demo': pd.read_csv(os.path.join(root_dir, f"DEMO{q}.txt"), delimiter='$', encoding='ISO-8859-1'),
/var/folders/1z/lz8g1zg14zv6dkhlyvsk7zvc0000gn/T/ipykernel_10022/1006456903.py:18: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  'drug': pd.read_csv(os.path.join(root_dir, f"DRUG{q}.txt"), delimiter='$', encoding='ISO-8859-1'),


Loaded 24Q4 successfully.


In [36]:
start_year = 2017 
start_quarter = 'Q2'
end_year = 2024
end_quarter = 'Q1'

demo, drug, reac, outc, indi, rpsr, ther = create_dataframes(start_year, start_quarter, end_year, end_quarter)


['17Q2', '17Q3', '17Q4', '18Q1', '18Q2', '18Q3', '18Q4', '19Q1', '19Q2', '19Q3', '19Q4', '20Q1', '20Q2', '20Q3', '20Q4', '21Q1', '21Q2', '21Q3', '21Q4', '22Q1', '22Q2', '22Q3', '22Q4', '23Q1', '23Q2', '23Q3', '23Q4', '24Q1']


In [37]:
demo.shape[0]
drug.shape[0]
reac.shape[0]

38957534

In [38]:
demo = preprocess_demo_df(demo)
# drug = preprocess_drug_df(drug)
# reac = preprocess_reac_df(reac)


In [39]:
query_drug = 'edaravone'

# Finds reports related to the query drug
query_drug_df = drug[
    drug['drugname'].str.lower().str.contains(query_drug, na=False) |
    drug['prod_ai'].str.lower().str.contains(query_drug, na=False)
]

In [40]:
query_drug_df_merged = pd.merge(demo, query_drug_df, on=['primaryid', 'caseid'], how='inner')
query_drug_df_merged

,primaryid,caseid,fda_dt,caseversion,age_cod,age,sex,wt,drug_seq,role_cod,...,cum_dose_unit,dechal,rechal,lot_num,exp_dt,nda_num,dose_amt,dose_unit,dose_form,dose_freq
0,63373272,6337327,20070705,2,YR,70.0,M,60.000,26,SS,...,NaN,NaN,NaN,NaN,NaN,NaN,30,MG,"INJECTION, SOLUTION",NaN
1,737231510,7372315,20181005,10,YR,71.0,F,NaN,12,C,...,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1165786410,11657864,20200715,10,YR,48.0,F,49.600,20,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,118102753,11810275,20170710,3,YR,75.0,M,53.000,5,C,...,MG,NaN,NaN,NaN,NaN,NaN,60.0,MG,NaN,QD
4,118163064,11816306,20170519,4,YR,81.0,F,NaN,4,SS,...,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4622,236749971,23674997,20240326,1,YR,34.0,F,NaN,2,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4623,236830131,23683013,20240328,1,YR,56.0,M,91.399,3,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4624,236830191,23683019,20240328,1,YR,37.0,M,NaN,4,SS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Suspension,NaN
4625,236882611,23688261,20240329,1,YR,73.0,F,NaN,5,C,...,NaN,NaN,NaN,NaN,NaN,NaN,30.0,MG,NaN,QD


In [41]:
# Descriptive stats: Get sex breakdown 
counts = query_drug_df_merged['sex'].value_counts()
percentages = query_drug_df_merged['sex'].value_counts(normalize=True) * 100

summary_df = pd.DataFrame({'count': counts, 'percent': percentages.round(2)})
summary_df

,count,percent
sex,,
M,1978,58.47
F,1404,41.50
UNK,1,0.03


In [47]:
# Get AE counts for cases where query drug is mentioned
query_drug_ids = query_drug_df_merged['primaryid'].unique()
query_drug_reac = reac[reac['primaryid'].isin(query_drug_ids)]
ae_counts = query_drug_reac['pt'].value_counts().reset_index()
ae_counts.columns = ['pt_name', 'Count']

# Get AE counts for cases where query drug is not mentioned
non_query_drug = drug[~drug['primaryid'].isin(query_drug_ids)]
non_query_ids = non_query_drug['primaryid'].unique()
non_query_reac = reac[reac['primaryid'].isin(non_query_ids)]
non_ae_counts = non_query_reac['pt'].value_counts().reset_index()
non_ae_counts.columns = ['pt_name', 'Count']

ae_counts.columns = ['pt_name', 'Count_query_drug']
non_ae_counts.columns = ['pt_name', 'Count_non_query_drug']

# Merge AE counts for query drug and non-query drug
ae_comparison = pd.merge(
    ae_counts,
    non_ae_counts,
    on='pt_name',
    how='outer'
)

# Filter to AEs with at least 3 reports for the query drug
ae_filtered = ae_comparison[
    (ae_comparison['Count_query_drug'].notna()) &
    (ae_comparison['Count_query_drug'] >= 3)
].copy().reset_index() 

In [48]:
# Calculates the number of reports that did not include a specific adverse
# event for both the query drug and non-query drugs
query_num = query_drug_ids.shape[0]
non_num = non_query_ids.shape[0]
ae_filtered['No_AE_query_drug'] = query_num - ae_filtered['Count_query_drug']
ae_filtered['No_AE_non_query_drug'] = non_num - ae_filtered['Count_non_query_drug']
ae_filtered = ae_filtered.drop('index', axis=1)

In [51]:
ae_filtered_new = add_stats(ae_filtered.copy())

                                      pt_name  Count_query_drug  \
0                        Abdominal discomfort              30.0   
1                        Abdominal distension               8.0   
2                              Abdominal pain              16.0   
3                        Abdominal pain upper              24.0   
4              Accidental exposure to product               3.0   
..                                        ...               ...   
352                          Weight increased               5.0   
353                                  Wheezing               6.0   
354          White blood cell count decreased               7.0   
355          White blood cell count increased               5.0   
356  Wrong technique in product usage process               3.0   

     Count_non_query_drug  No_AE_query_drug  No_AE_non_query_drug  odds_ratio  \
0                  116650            3588.0              11833297    0.848184   
1                   62060        

In [ ]:
ae_filtered_new.sort_values(by='p_value')
ae_filtered_new.sort_values(by='Count_query_drug').to_csv(f"pt_terms_{query_drug}.csv", index=False)


In [ ]:
root_dir = '/Users/jodie/Documents/BMI 212/faers-cohort-generation'
llt_soc = pd.read_csv(os.path.join(root_dir, 'llt_soc.csv'))

In [ ]:
soc_filtered = (
    ae_filtered
    .merge(llt_soc[['pt_name', 'soc_name']], how='left', on='pt_name')
    .groupby('soc_name', as_index=False)
    .agg({
        'Count_query_drug': 'sum',
        'Count_non_query_drug': 'sum',
        'No_AE_query_drug': 'sum',
        'No_AE_non_query_drug': 'sum'
    })
)


In [ ]:
soc_filtered_new = add_stats(soc_filtered.copy())

In [ ]:
soc_filtered_new.sort_values(by='Count_query_drug').to_csv(f"soc_terms_{query_drug}.csv", index=False)